# Cochran Q test
*By P. Stikker*<br>
https://PeterStatistics.com<br>
https://www.youtube.com/stikpet<br>

## Introduction

If you allowed people to choose more than one option (e.g. which of the following music genres do you like?), each of the options becomes a no/yes variable in itself, which are binary variables. It might be that not every option was chosen equally often. To test if these differences are significantly different we can use a Cochran's Q test.

The Cochran's Q test starts similar as a Pearson chi-square test by squaring the differences between the observed and the expected proportions, but then divides this by the sum of the number of successes multiplied by the number of failures for each case.

Lets see how we can do this test with Python.

## Example

To show an example, I'll load some data as a pandas dataframe. So I'll need the '<a href="https://pandas.pydata.org">pandas</a>' library:

In [4]:
#!pip install pandas
import pandas as pd

And then load the example data using the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">'read_csv'</a>. 

In [8]:
myDf = pd.read_csv('FilmPreferences2123.csv')
myDf.head()

,sex,age,educat,income,thriller,horror,comedy,adventur,docu,roman,munt,movies,tuschin,arena
0,0.0,25.0,2.0,2000.0,6.0,3.0,6.0,6.0,9.0,6.0,0.0,0.0,0.0,0.0
1,0.0,33.0,2.0,2350.0,4.0,2.0,7.0,8.0,4.0,7.0,0.0,0.0,0.0,1.0
2,0.0,44.0,3.0,4500.0,6.0,3.0,7.0,8.0,8.0,7.0,1.0,1.0,1.0,0.0
3,0.0,53.0,3.0,5800.0,4.0,1.0,5.0,9.0,7.0,4.0,1.0,1.0,1.0,0.0
4,0.0,64.0,2.0,4450.0,5.0,4.0,6.0,9.0,9.0,8.0,1.0,0.0,0.0,0.0


The fields *munt*, *movies*, *tushin*, and *arena* are different cinemas (in Amsterdam), where a 1 would indicate the respondent visited the cinema, and a 0 if they didn't. They could select more than one. 

Lets combine these in their own dataframe.

In [23]:
selData = myDf[['munt', 'movies', 'tuschin', 'arena']]
selData.head()

,munt,movies,tuschin,arena
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0
2,1.0,1.0,1.0,0.0
3,1.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0


The Cochran's Q test is available in the statsmodel.api package, so lets load its function:

In [10]:
# !pip install statsmodels.api
from statsmodels.stats.contingency_tables import cochrans_q

To see the results, we print the function:

In [11]:
print(cochrans_q(selData))

df          3
pvalue      0.00022763307854354147
statistic   19.384615384615383


In the example the Cochran's Q value is 19.38 and has a significance of .000228. This indicates that there is a very low chance (.000228 is usually considered low, the threshold is usually set at .05 or lower) to obtain a Q value of 19.38 or even higher in a sample, if in the population there would be no differences. This is so low that most likely in the populations each variable is not chosen equally often.

If you are interested, in the appendix I'll go over the formulas and avoid packages (except for the chi-square distribution that is needed for the p-value).

## Appendix: The Hard Way

First, convert the data into something that is native for Python: a list.

In [12]:
selData = [list(myDf['munt'])] + [list(myDf['movies'])] + [list(myDf['tuschin'])] + [list(myDf['arena'])]

Some basic variables can be helpful. 

*n* as the number of cases, and *k* as the number of variables/fields. Each can easily be found using the len function of Python

In [13]:
n = len(selData[0])
k = len(selData)

n,k

(150, 4)

Now we determine the column totals for each field, i.e. how many respondents opted for each field. It is important that the data is coded with 0 and 1.

In [14]:
C = []
for i in range(k):
    C = C + [sum(selData[i])]
    
C

[81.0, 45.0, 61.0, 57.0]

We can also sum all these ($n_c$) to see how many cinemas were visited in total:

In [15]:
nc = sum(C)
nc

244.0

Next is the sum of squares for these. This is defined as:

\begin{equation*}
SS_c = \sum_{i=1}^k \left( C_i - \frac{n_c} {k} \right)^2
\end{equation*}


In [16]:
SSc = 0
for i in range(k):
    SSc = SSc + (C[i] - nc/k)**2
    
SSc    

672.0

The standard error (not really sure if this is the official name, but okay):

\begin{equation*}
SE = \sum_{i=1}^n R_i \times \left( k - R_i\right)
\end{equation*}

Where $R_i$ is the row sum of the i-th case (so in the example how many cinemas the respondent had visited in total).

In [17]:
SE = 0
for i in range(n):
    rSum = 0
    for j in range(k):
        rSum = rSum + selData[j][i]
    SE = SE + rSum*(k - rSum)

SE

416.0

Finally the test statistic can be calculated using:

\begin{equation*}
Q = k \times \left( k - 1 \right) \times \frac{SS_c} {SE}
\end{equation*}


In [18]:
Q = k * (k - 1) * SSc / SE
Q

19.384615384615383

This statistic follows a chi-square distribution with degrees of freedom as:

\begin{equation*}
df = k - 1
\end{equation*}

In [19]:
df = k - 1
df

3

To find the corresponding p-value we need the chi-square distribution. For this a package will be used. Lets use scipy.stats

In [20]:
from scipy.stats import chi2

Finally the p-value can be found:

In [21]:
chi2.sf(Q, df)

0.00022763307854354147